In [38]:
import re
import pandas as pd
%matplotlib inline

In [39]:
article_user_data = 'data/article_user.csv'
article_user = pd.read_csv(article_user_data)
articles_data = 'data/articles.csv'
articles = pd.read_csv(articles_data)

In [40]:
#articles.head().to_csv('articles_sample.csv')

In [41]:
articles.head(2)

,article_id,headline,url,published_date
0,e9d06b2bb42bd9970de43beab2d3893534d7a619,Defense Department slams brakes on border wall...,https://www.cnn.com/2021/01/21/politics/border...,2021-01-21 21:45:03.000 Z
1,e98b0d02b16794bc0884a61c4a45746cb9430630,Biden's DOJ must confront what to do with outs...,https://www.cnn.com/2021/01/21/politics/donald...,2021-01-22 02:11:02.000 Z


In [42]:
len(article_user.user_id.unique())

50000

In [43]:
len(articles.article_id.unique())

1455

In [44]:
articles.head().to_csv('articles_sample.csv')

In [45]:
article_user.head(2)

,user_id,article_id,timestamp
0,2bc424123e0a12d29c488bb6e565fe98d0a49b46,e9d06b2bb42bd9970de43beab2d3893534d7a619,1611291180
1,2bc424123e0a12d29c488bb6e565fe98d0a49b46,e98b0d02b16794bc0884a61c4a45746cb9430630,1611290464


In [46]:
df = pd.merge(article_user, articles)

In [47]:
#pd.to_datetime(df['timestamp'], infer_datetime_format=True)  

In [48]:
from datetime import datetime, timedelta
def convert_time(time_str):
    ts = int(time_str)
    return datetime.utcfromtimestamp(ts)

In [49]:
df['timestamp'] = df.apply(lambda r: convert_time(r['timestamp']), axis=1)

In [50]:
datetime.strptime('2021-04-19', '%Y-%m-%d') - timedelta(days=7)

datetime.datetime(2021, 4, 12, 0, 0)

In [51]:
f1 = df['timestamp'] >=  datetime.now() - timedelta(days=120)
f2 = df['timestamp'] <  datetime.now() - timedelta(days=100)
df[f2]

,user_id,article_id,timestamp,headline,url,published_date


In [52]:
df['timestamp'].max()

Timestamp('2021-01-25 00:09:41')

In [53]:
df['timestamp'].min()

Timestamp('2021-01-20 00:00:31')

In [54]:
datetime.now() - timedelta(days=100)

datetime.datetime(2021, 1, 12, 21, 50, 59, 191141)

In [55]:
datetime.now() - timedelta(days=120)

datetime.datetime(2020, 12, 23, 21, 50, 59, 197282)

In [56]:
# Most popular score

most popular articles are something. What are they?

In [57]:
#df['article_id'].value_counts().to_csv('article_counts.csv')

In [58]:
def get_topic(url):
    pattern = 'http(s)?://(www|money).cnn.com/\d{4}/\d{2}/\d{2}/(?P<topic>[a-z0-9-]+)/'
    result = re.match(pattern, url)
    topic = None
    if result:
        topic = result.group('topic')
    if not result:
        pattern2 = 'https://www.cnn.com/(?P<topic>[a-z]*)/'
        result = re.match(pattern2, url)
        if result:
            topic = result.group('topic')
    if topic:
        return topic
    else:
        return ''

In [67]:
df['category'] = df.apply(lambda r: get_topic(r['url']), axis=1)

In [68]:
# # calculate the frequency of each article
groups = df.groupby(['category', 'article_id']).size().reset_index().rename({0: 'frequency'}, axis=1)
# n = 5
# # get the top n articles
groups = groups.sort_values('frequency', ascending=False).groupby('category')\
            .head(10).sort_values(['category', 'frequency'], ascending=False)

In [69]:
msk1 = groups['category'] == 'world'
groups[msk1].head(5)

,category,article_id,frequency
1431,world,889f731b80b1becc703d0f550da3fe85be9251eb,1518
1444,world,d3810241ac4c40506c1392a189fd67abf0d9baf8,1407
1440,world,c0587cef8661f800b8f47c8af6520692fd45a1ad,464
1437,world,a1981821adab54be5325ee596c350bf3ea0ae370,426
1420,world,35a820c34b6623bf91e196cad2b1cf49342ece42,293


In [70]:
categories = groups['category'].unique()
def category_exists(category: str) -> bool:
    return category in categories

def get_top_n_in_category(n, category) -> pd.DataFrame:
    n = max(5, n)
    msk1 = groups['category'] == category
    return groups[msk1].head(n)

In [71]:
# groups.to_csv('nlargest.csv')

In [72]:
all_articles = pd.merge(article_user, articles)

In [73]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [74]:
%time
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(articles['headline'].unique())

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 5.25 µs


In [75]:
tfidf_matrix.shape

(1455, 13246)

In [76]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [77]:
all_articles['headline'][0]

'Defense Department slams brakes on border wall as it reviews Biden order'

In [78]:
uid = '2bc424123e0a12d29c488bb6e565fe98d0a49b46'
all_articles[all_articles['user_id'] == '2bc424123e0a12d29c488bb6e565fe98d0a49b46'].head(2)

,user_id,article_id,timestamp,headline,url,published_date
0,2bc424123e0a12d29c488bb6e565fe98d0a49b46,e9d06b2bb42bd9970de43beab2d3893534d7a619,1611291180,Defense Department slams brakes on border wall...,https://www.cnn.com/2021/01/21/politics/border...,2021-01-21 21:45:03.000 Z
714,2bc424123e0a12d29c488bb6e565fe98d0a49b46,e98b0d02b16794bc0884a61c4a45746cb9430630,1611290464,Biden's DOJ must confront what to do with outs...,https://www.cnn.com/2021/01/21/politics/donald...,2021-01-22 02:11:02.000 Z


In [79]:
smd = all_articles.reset_index()
headlines = articles['headline']
indices = pd.Series(articles.index, index=articles['headline'])
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return headlines.iloc[movie_indices]

In [80]:
t = 'Defense Department slams brakes on border wall as it reviews Biden order'
get_recommendations(t).head(5)#.to_csv('similar_recommendations.csv')

1223    Biden's desire to stop the border wall could b...
908     Steve Bannon, three others charged with fraud ...
483     Trump visits his border wall on the heels of d...
1239    Trump administration locks down border wall co...
169     Biden starts fast on immigration by halting bo...
Name: headline, dtype: object

In [81]:
all_articles['rating'] = 1

In [93]:
msk3 = all_articles['user_id'] == '2bc424123e0a12d29c488bb6e565fe98d0a49b46'
all_articles[msk3].head(2)

,user_id,article_id,timestamp,headline,url,published_date,rating
0,2bc424123e0a12d29c488bb6e565fe98d0a49b46,e9d06b2bb42bd9970de43beab2d3893534d7a619,1611291180,Defense Department slams brakes on border wall...,https://www.cnn.com/2021/01/21/politics/border...,2021-01-21 21:45:03.000 Z,1
714,2bc424123e0a12d29c488bb6e565fe98d0a49b46,e98b0d02b16794bc0884a61c4a45746cb9430630,1611290464,Biden's DOJ must confront what to do with outs...,https://www.cnn.com/2021/01/21/politics/donald...,2021-01-22 02:11:02.000 Z,1


In [83]:
g = df.groupby(['user_id', 'article_id'])['article_id'].count()#.to_frame()
#ratings = g.rename('rating').to_frame().reset_index()
article_counts = g.rename('rating').to_frame().reset_index()

In [94]:
#ratings[ratings['rating'] > 1].head(2)

In [96]:
article_counts.head(2)

,user_id,article_id,rating
0,0002604d5804123dedb91819acbc93f888c54b8e,534c4076ad58bee6998339f70a018a0330cdcbf5,1
1,0002604d5804123dedb91819acbc93f888c54b8e,6ab428f68d0fa9a6405a508751acc8ef0b9ee4ff,1


In [98]:
number_of_visited_articles = df.groupby(['user_id'])['article_id'].count().to_frame().reset_index().rename({'article_id': 'number_of_visited_articles'}, axis=1)
number_of_visited_articles.head(2)

,user_id,number_of_visited_articles
0,0002604d5804123dedb91819acbc93f888c54b8e,6
1,0003728d0245028f76f7a655241a07181d1be815,20


In [99]:
article_counts_merged = pd.merge(number_of_visited_articles, article_counts)

In [100]:
article_counts_merged['normalized_rating'] = article_counts_merged['rating'] / article_counts_merged['number_of_visited_articles']

In [101]:
# add normalized ratings
df = pd.merge(df, article_counts_merged.drop(['rating'], axis=1))

In [102]:
p = ratings.groupby(['user_id', 'article_id']).sum()['rating'].to_frame()
p[p['rating'] > 1]

NameError: name 'ratings' is not defined

In [103]:
import surprise
from surprise import Reader, Dataset, SVD, model_selection#, evaluate
svd = SVD()
reader = Reader()
data = Dataset.load_from_df(df[['user_id', 'article_id', 'normalized_rating']], reader)
model_selection.cross_validate(svd, data, measures=['RMSE', 'MAE'])

{'test_rmse': array([0.88733303, 0.88768809, 0.88678923, 0.88732183, 0.88742525]),
 'test_mae': array([0.88206704, 0.88250538, 0.88150491, 0.88209109, 0.88217919]),
 'fit_time': (19.19094204902649,
  19.284076690673828,
  19.191633224487305,
  19.223021984100342,
  19.227370977401733),
 'test_time': (0.7593529224395752,
  0.5924577713012695,
  0.6702642440795898,
  0.7498610019683838,
  0.6855781078338623)}

In [104]:
trainset = data.build_full_trainset()
svd.fit(trainset)

In [105]:
svd.predict('0002604d5804123dedb91819acbc93f888c54b8e', 'fffb5887fbd6f820ca50ce3787fdd8bd2c87a3c4', 3)

Prediction(uid='0002604d5804123dedb91819acbc93f888c54b8e', iid='fffb5887fbd6f820ca50ce3787fdd8bd2c87a3c4', r_ui=3, est=1, details={'was_impossible': False})

In [106]:
def get_recommendations_from_user_id_headline(user_id, headline):
    idx = indices[headline]
    sim_scores = list(enumerate(cosine_sim[int(idx)]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    article_indices = [i[0] for i in sim_scores]
    
    articles_tmp = articles.iloc[article_indices][['headline', 'article_id']]
    articles_tmp['est'] = articles_tmp.apply(lambda x: svd.predict(user_id, x['article_id']).est, axis=1)
    articles_tmp = articles_tmp.sort_values('est', ascending=False)
    return articles.head(5)

In [107]:
articles.head(2)

,article_id,headline,url,published_date
0,e9d06b2bb42bd9970de43beab2d3893534d7a619,Defense Department slams brakes on border wall...,https://www.cnn.com/2021/01/21/politics/border...,2021-01-21 21:45:03.000 Z
1,e98b0d02b16794bc0884a61c4a45746cb9430630,Biden's DOJ must confront what to do with outs...,https://www.cnn.com/2021/01/21/politics/donald...,2021-01-22 02:11:02.000 Z


In [108]:
smd.head(2)

,index,user_id,article_id,timestamp,headline,url,published_date
0,0,2bc424123e0a12d29c488bb6e565fe98d0a49b46,e9d06b2bb42bd9970de43beab2d3893534d7a619,1611291180,Defense Department slams brakes on border wall...,https://www.cnn.com/2021/01/21/politics/border...,2021-01-21 21:45:03.000 Z
1,1,b5c93b1ce2245fe8a4d59aedbd6c33a8b315bf60,e9d06b2bb42bd9970de43beab2d3893534d7a619,1611255943,Defense Department slams brakes on border wall...,https://www.cnn.com/2021/01/21/politics/border...,2021-01-21 21:45:03.000 Z


In [109]:
user_id = '2bc424123e0a12d29c488bb6e565fe98d0a49b46'
headline = 'Defense Department slams brakes on border wall as it reviews Biden order'
get_recommendations_from_user_id_headline(user_id, headline)

# can't find this: 'article_id'

,article_id,headline,url,published_date
0,e9d06b2bb42bd9970de43beab2d3893534d7a619,Defense Department slams brakes on border wall...,https://www.cnn.com/2021/01/21/politics/border...,2021-01-21 21:45:03.000 Z
1,e98b0d02b16794bc0884a61c4a45746cb9430630,Biden's DOJ must confront what to do with outs...,https://www.cnn.com/2021/01/21/politics/donald...,2021-01-22 02:11:02.000 Z
2,05f51c586ada891448ab56d9a5c8adb2a688e2fb,White House wants Democrats to be patient on s...,https://www.cnn.com/2021/01/22/politics/covid-...,2021-01-22 21:53:55.000 Z
3,91333a3c6d28c9241cbb01678a97699232b79e70,The Supreme Court's run out of excuses to avoi...,https://www.cnn.com/2021/01/24/politics/suprem...,2021-01-24 18:42:26.000 Z
4,30170d08779c54b84170d4cb3b5cbe0462df704a,Evidence shows Capitol rioters brutally attack...,https://www.cnn.com/2021/01/21/politics/capito...,2021-01-21 12:45:30.000 Z


In [110]:
for a in get_recommendations_from_user_id_headline(user_id, headline):
    print(a)

article_id
headline
url
published_date


In [111]:
def get_user_history_headlines(user_id: str) -> set:
    msk = all_articles['user_id'] == user_id
    headlines = all_articles[msk]['headline'].unique()
    return headlines

In [112]:
get_user_history_headlines(user_id)

array(['Defense Department slams brakes on border wall as it reviews Biden order',
       "Biden's DOJ must confront what to do with outstanding cases in which it is defending Trump. The first test could come Friday.",
       'White House wants Democrats to be patient on stimulus talks as Biden pushes for bipartisan path, officials say',
       "The Supreme Court's run out of excuses to avoid controversial issues",
       'Evidence shows Capitol rioters brutally attacked police with flagpoles, fire extinguishers and fists',
       'Four presidents take a stand',
       'I went to Washington for joy, and Amanda Gorman delivered it',
       "Syroco vs SP80: The race to create the world's fastest sail boat",
       'Here are the 30 executive orders and actions Biden signed in his first three days',
       "Joe Biden grieves Covid victims on eve of his inauguration: 'To heal, we must remember'",
       'Putin presents a Russia-sized foreign-policy headache for Biden',
       'Why Republica

In [113]:
user_id

'2bc424123e0a12d29c488bb6e565fe98d0a49b46'

In [119]:
dfs = []
for i, headline in enumerate(get_user_history_headlines(user_id)):
    user_history_headlines = get_user_history_headlines(user_id)
    for headline in user_history_headlines:
        try:
            recs = get_recommendations_from_user_id_headline(user_id, headline)
            dfs.append(recs)
        except:
            print('user_id', user_id)
            print('headline', headline)
        

In [120]:
pd.concat(dfs)

,article_id,headline,url,published_date
0,e9d06b2bb42bd9970de43beab2d3893534d7a619,Defense Department slams brakes on border wall...,https://www.cnn.com/2021/01/21/politics/border...,2021-01-21 21:45:03.000 Z
1,e98b0d02b16794bc0884a61c4a45746cb9430630,Biden's DOJ must confront what to do with outs...,https://www.cnn.com/2021/01/21/politics/donald...,2021-01-22 02:11:02.000 Z
2,05f51c586ada891448ab56d9a5c8adb2a688e2fb,White House wants Democrats to be patient on s...,https://www.cnn.com/2021/01/22/politics/covid-...,2021-01-22 21:53:55.000 Z
3,91333a3c6d28c9241cbb01678a97699232b79e70,The Supreme Court's run out of excuses to avoi...,https://www.cnn.com/2021/01/24/politics/suprem...,2021-01-24 18:42:26.000 Z
4,30170d08779c54b84170d4cb3b5cbe0462df704a,Evidence shows Capitol rioters brutally attack...,https://www.cnn.com/2021/01/21/politics/capito...,2021-01-21 12:45:30.000 Z
...,...,...,...,...
0,e9d06b2bb42bd9970de43beab2d3893534d7a619,Defense Department slams brakes on border wall...,https://www.cnn.com/2021/01/21/politics/border...,2021-01-21 21:45:03.000 Z
1,e98b0d02b16794bc0884a61c4a45746cb9430630,Biden's DOJ must confront what to do with outs...,https://www.cnn.com/2021/01/21/politics/donald...,2021-01-22 02:11:02.000 Z
2,05f51c586ada891448ab56d9a5c8adb2a688e2fb,White House wants Democrats to be patient on s...,https://www.cnn.com/2021/01/22/politics/covid-...,2021-01-22 21:53:55.000 Z
3,91333a3c6d28c9241cbb01678a97699232b79e70,The Supreme Court's run out of excuses to avoi...,https://www.cnn.com/2021/01/24/politics/suprem...,2021-01-24 18:42:26.000 Z
